### TODO: unfinished

In [1]:
# !git clone https://github.com/myshell-ai/OpenVoice

In [1]:
import os
import torch
import openvino as ov
core = ov.Core()

In [2]:
ov.__file__

'/home/epavel/devel/openvino/tools/mo/openvino/__init__.py'

In [3]:
# cd to the original repo to save original data paths and imports
%cd OpenVoice

/home/epavel/devel/openvino_notebooks/notebooks/280-openvoice/OpenVoice


/home/epavel/opt/envs/py310-openvoice/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


download all resources from HF Hub

In [4]:
from notebook_utils import download_file
base_url = 'https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/'

CKPT_BASE_PATH = '../checkpoints/'
en_ckpt_path = f'{CKPT_BASE_PATH}/base_speakers/EN/'
zh_ckpt_path = f'{CKPT_BASE_PATH}/base_speakers/ZH/'
converter_path = f'{CKPT_BASE_PATH}/converter/'

In [5]:
from api import BaseSpeakerTTS, ToneColorConverter, OpenVoiceBaseClass
import se_extractor

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [6]:
device="cuda:0" if torch.cuda.is_available() else "cpu"

en_base_speaker_tts = BaseSpeakerTTS(f'{en_ckpt_path}/config.json', device=device)
en_base_speaker_tts.load_ckpt(f'{en_ckpt_path}/checkpoint.pth')
tone_color_converter = ToneColorConverter(f'{converter_path}/config.json', device=device)
tone_color_converter.load_ckpt(f'{converter_path}/checkpoint.pth')

/home/epavel/opt/envs/py310-openvoice/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint '../checkpoints//base_speakers/EN//checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint '../checkpoints//converter//checkpoint.pth'
missing/unexpected keys: [] []


### Convert models to OpenVINO

To convert to OpenVino IR format first we need to get acceptable pytorch nn.Module object. 

Both ToneColorConverter, BaseSpeakerTTS instead of using self.forward as the main entry point use custom methods infer and convert_voice respectively, therefore need to wrap them with a custom class that is inherited from torch.nn.Module. 

<!---
# One more reason to make a wrapper is also that these functions use float arguments while only torch.Tensor and tuple of torch.Tensors are acceptable 
# todo: check if it works when kwargs are moved to example inputs.
-->

In [7]:
tts_kwargs = dict(noise_scale = 0.667, noise_scale_w = 0.6, speed = 1.0, sdp_ratio = 0.2)
voice_convert_kwargs = dict(tau=0.3)

class OVOpenVoiceBase(torch.nn.Module):
    def __init__(self, voice_model: OpenVoiceBaseClass, **kwargs):
        super().__init__()
        self.voice_model = voice_model
        self.default_kwargs = kwargs
        for par in voice_model.model.parameters():
            par.requires_grad = False
    
class OVOpenVoiceTTS(OVOpenVoiceBase):
    def get_example_input(self):
        stn_tst = self.voice_model.get_text('this is original text', self.voice_model.hps, False)
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        speaker_id = torch.LongTensor([1])
        return (x_tst, x_tst_lengths, speaker_id)

    def forward(self, x, x_lengths, sid):
        return self.voice_model.model.infer(x, x_lengths, sid, **self.default_kwargs)
    
class OVOpenVoiceConverter(OVOpenVoiceBase):
    def get_example_input(self):
        y = torch.randn([1, 513, 238], dtype=torch.float32)
        y_lengths = torch.LongTensor([y.size(-1)])
        target_se = torch.randn(*(1, 256, 1))
        source_se = torch.randn(*(1, 256, 1))
        return (y, y_lengths, source_se, target_se)
    
    def forward(self, y, y_lengths, sid_src, sid_tgt):
        return self.voice_model.model.voice_conversion(y, y_lengths, sid_src, sid_tgt, **self.default_kwargs)

Convert to OpenVino IR and save to IRs_path folder for the future use. If IRs already exist skip conversion and read them directly

In [8]:
IRS_PATH = '../openvino_irs/'
TTS_IR = f'{IRS_PATH}/openvoice_tts.xml'
VOICE_CONVERTER_IR = f'{IRS_PATH}/openvoice_tone_conversion.xml'

if not os.path.exists(TTS_IR):
    pt_tts = OVOpenVoiceTTS(en_base_speaker_tts)
    ov_tts = ov.convert_model(pt_tts, example_input=pt_tts.get_example_input())
    ov.save_model(ov_tts, TTS_IR)
else:
    ov_tts = core.read_model(TTS_IR)

if not os.path.exists(VOICE_CONVERTER_IR):
    pt_voice_converter = OVOpenVoiceConverter(tone_color_converter)
    ov_voice_conversion = ov.convert_model(pt_voice_converter, example_input=pt_voice_converter.get_example_input())
    ov.save_model(ov_voice_conversion, VOICE_CONVERTER_IR)
else:
    ov_voice_conversion = core.read_model(VOICE_CONVERTER_IR)

In [9]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='CPU',
    description='Device:',
    disabled=False,
)
device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

First of all, select the reference tone of voice to which the generated text will be converted: your can select from existing ones or record your own by seleceing 'record_manually'

In [10]:
reference_speakers = [
    'resources/example_reference.mp3',
    'resources/demo_speaker0.mp3',
    'resources/demo_speaker1.mp3',
    'resources/demo_speaker2.mp3',
    'record_manually',
]

ref_speaker = widgets.Dropdown(
    options=reference_speakers,
    value=reference_speakers[0],
    description="reference voice from which tone color will be copied",
    disabled=False,
)

display(ref_speaker)

Dropdown(description='reference voice from which tone color will be copied', options=('resources/example_refer…

In [11]:
output_dir = '../outputs/'
os.makedirs(output_dir, exist_ok=True)

In [12]:
ref_speaker_path = ref_speaker.value

if ref_speaker.value == 'record_manually':
    ref_speaker_path = f'{output_dir}/custom_example_sample.webm'
    from ipywebrtc import AudioRecorder, CameraStream
    camera = CameraStream(constraints={'audio': True,'video':False})
    recorder = AudioRecorder(stream=camera, filename=ref_speaker_path, autosave=True)
    display(recorder)

In [13]:
from IPython.display import Audio
Audio(ref_speaker_path)

In [14]:
# load speaker embeddings
en_source_default_se = torch.load(f'{en_ckpt_path}/en_default_se.pth')
en_source_style_se = torch.load(f'{en_ckpt_path}/en_style_se.pth')
zh_source_se = torch.load(f'{zh_ckpt_path}/zh_default_se.pth')

target_se, audio_name = se_extractor.get_se(ref_speaker_path, tone_color_converter, target_dir='processed', vad=True)  ## ffmpeg must be installed

custom_se = se_extractor.get_se('/home/epavel/my_base_voice.m4a', tone_color_converter, target_dir='processed', vad=True)[0]  ## ffmpeg must be installed

/home/epavel/opt/envs/py310-openvoice/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


### Inference

There are pre and post processings that are not traceable and could not be offloaded to OpenVINO, instead of writing such processing ourselves we will rely on the already existing ones. We just replace infer and voice conversion functions of OpenVoiceBaseClass so that the the most computationally expensive part is done in OpenVINO.

In [15]:
def get_pathched_infer(ov_model: ov.Model, device_name: str = device.value) -> callable:
    compiled_model = core.compile_model(ov_model, device_name)
    
    def infer_impl(x, x_lengths, sid=None, noise_scale=1, length_scale=1, noise_scale_w=1., sdp_ratio=0.2, max_len=None):
        # todo: assert that other params match to compiled ones
        ov_output = compiled_model((x, x_lengths, sid))
        return (torch.tensor(ov_output[0]), )
    return infer_impl

def get_patched_voice_conversion(ov_model: ov.Model, device_name: str = device.value) -> callable:
    compiled_model = core.compile_model(ov_model, device_name)

    def voice_conversion_impl(y, y_lengths, sid_src, sid_tgt, tau=1.0):
        # todo: assert that tau matches to compiled ones
        ov_output = compiled_model((y, y_lengths, sid_src, sid_tgt))
        return (torch.tensor(ov_output[0]), )
    return voice_conversion_impl

en_base_speaker_tts.model.infer = get_pathched_infer(ov_tts)
tone_color_converter.model.voice_conversion = get_patched_voice_conversion(ov_voice_conversion)

In [16]:
article = """
The work chronicles the Napoleonic era within Russia, 
notably detailing the French invasion of Russia and its aftermath. 
The book highlights the impact of Napoleon on Tsarist society 
through five interlocking narratives following different 
Russian aristocratic families. Portions of an earlier version, 
titled The Year 1805, were serialized in The Russian Messenger 
from 1865 to 1867 before the novel was published in its entirety in 1869.
Tolstoy said that the best Russian literature does not conform 
to standards and hence hesitated to classify War and Peace, 
saying it is "not a novel, even less is it a poem, 
and still less a historical chronicle". Large sections, 
especially the later chapters, are philosophical discussions rather 
than narrative. He regarded Anna Karenina as his first true novel.
"""

In [17]:
lines = en_base_speaker_tts.split_sentences_into_pieces(article, 'EN')

 > Text splitted to sentences.
The work chronicles the Napoleonic era within Russia, notably detailing the French invasion of Russia and its aftermath.
The book highlights the impact of Napoleon on Tsarist society through five interlocking narratives following different Russian aristocratic families.
Portions of an earlier version, titled The Year 1805, were serialized in The Russian Messenger from 1865 to 1867 before the novel was published in its entirety in 1869.
Tolstoy said that the best Russian literature does not conform to standards and hence hesitated to classify War and Peace,
saying it is not a novel, even less is it a poem,
and still less a historical chronicle. Large sections, especially the later chapters,
are philosophical discussions rather than narrative. He regarded Anna Karenina as his first true novel.
 > ===========================


In [18]:
pt_device = 'cpu'

In [19]:
import nncf
import re
mark = 'EN'
speaker='default'

def transform_fn(line: str):
    t = re.sub(r'([a-z])([A-Z])', r'\1 \2', line)
    t = f'[{mark}]{t}[{mark}]'
    stn_tst = en_base_speaker_tts.get_text(t, en_base_speaker_tts.hps, False)
    speaker_id = en_base_speaker_tts.hps.speakers[speaker]
    x_tst = stn_tst.unsqueeze(0).to(pt_device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(pt_device)
    sid = torch.LongTensor([speaker_id]).to(pt_device)
    return (x_tst, x_tst_lengths, sid)

calibration_dataset = nncf.Dataset(lines, transform_fn)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [20]:
ignored_scope = nncf.IgnoredScope(
    types=["Multiply", "Subtract", "Sigmoid"],  # ignore operations
)

# quantized_model = nncf.quantize(ov_tts, calibration_dataset, ignored_scope=ignored_scope)
# ov.save_model(quantized_model, '../openvino_irs/openvoice_tts_quantized.xml')

quantized_model = ov.Core().read_model('../openvino_irs/openvoice_tts_quantized.xml')

In [25]:
en_base_speaker_tts.model.infer = get_pathched_infer(quantized_model)

In [21]:
ov.__file__

'/home/epavel/devel/openvino/tools/mo/openvino/__init__.py'

In [26]:
save_path = f'{output_dir}/output_en_default.wav'

text = """
OpenVINO toolkit is a comprehensive toolkit for quickly developing applications and solutions that solve 
a variety of tasks including emulation of human vision, automatic speech recognition, natural language processing, 
recommendation systems, and many others.
"""

src_path = f'{output_dir}/tmp.wav'
en_base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)
# src_path = '/home/epavel/my_base_voice.m4a'

tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=en_source_default_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message="@MyShell")

 > Text splitted to sentences.
OpenVINO toolkit is a comprehensive toolkit for quickly developing applications and solutions that solve a variety of tasks including emulation of human vision,
automatic speech recognition, natural language processing, recommendation systems, and many others.
 > ===========================
ˈoʊpən vino* toolkit* ɪz ə ˌkɑmpɹiˈhɛnsɪv toolkit* fəɹ kˈwɪkli dɪˈvɛləpɪŋ ˌæpləˈkeɪʃənz ənd səˈluʃənz ðət sɑɫv ə vəɹˈaɪəti əv tæsks ˌɪnˈkludɪŋ ˌɛmjəˈleɪʃən əv ˈjumən ˈvɪʒən,
 length:173
 length:173


RuntimeError: Exception from src/inference/src/infer_request.cpp:233:
Exception from src/plugins/intel_cpu/src/memory_desc/cpu_memory_desc.h:89:
ParameterMismatch: Can not clone with new dims. Descriptor's shape: {0 - ?, 192, 1} is incompatible with provided dimensions: {1, 192, 347}.



In [24]:
Audio(src_path)

In [32]:
Audio(save_path)

### Run OpenVoice Gradio online app
We can also use [Gradio](https://www.gradio.app/) app to run TTS and voice tone conversion online.

In [18]:
from openvoice_gradio import get_demo

demo = get_demo(output_dir, color_convert_model, en_tts_model, zh_tts_model, en_source_default_se, en_source_style_se, zh_source_se)
demo.queue(max_size=2)
demo.launch(server_name="0.0.0.0", server_port=7860)

/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/gradio/components/dropdown.py:90: UserWarning: The `max_choices` parameter is ignored when `multiselect` is False.
  warnings.warn(


Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


In [19]:
# please run this cell for stopping gradio interface
demo.close()

# clean up 
# import shutil
# shutil.rmtree(CKPT_BASE_PATH)
# shutil.rmtree(IRS_PATH)

Closing server running on port: 7860
